In [1]:

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score


from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

import sys
sys.path.append("../../")
from src.soporte_metricas import (
    obtener_metricas,
    comparar_arboles
)
pd.options.display.float_format = "{:,.2f}".format
# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("../../datos/04_Modelo4/06_autorenew_target_encoded.csv",index_col=0)
df.sample()

,offerType,brand,vehicleType,fuelType,gearbox,notRepairedDamage,kilometer,price,yearOfRegistration,powerCV_robust_scaler
292928,"6,065.12","8,533.32","5,252.19","5,152.20","10,743.01","7,212.17","3,822.65","1,490.00","12,583.66","6,113.02"


# Decision Tree
- El árbol de decisiones se usa para modelos no lineales
- Vamos a probarlo a ver si mejora nuestras métricas

In [6]:
X = df.drop(columns="price")
y = df[["price"]]
X_train, X_test, y_train, y_test = train_test_split(X, # Todo menos la Variable Respuesta
                                                    y, # La variable Respuesta
                                                    train_size=0.7, # Como dividir los datos en este caso 80% train, 20% test
                                                    random_state=42, # Semilla, para que lo haga igual para cualquier persona
                                                    shuffle=True 
                                                    )

In [7]:
modelo_arbol = DecisionTreeRegressor()
modelo_arbol.fit(X_train,y_train)
y_pred_test = modelo_arbol.predict(X_test)
y_pred_train = modelo_arbol.predict(X_train)
df_metricas = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricas

,r2_score,MAE,MSE,RMSE
train,0.87,873.48,"28,168,042.78","5,307.36"
test,0.28,"1,856.75","105,569,611.64","10,274.71"


In [8]:
modelo_arbol.get_depth()

35

In [14]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [10,15,20],
    'max_leaf_nodes': [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45],
    #'min_samples_leaf': [0, 1, 2, 3, 4, 5, 6, 7],
    'min_samples_split': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
    
}

grid_search_arbol = GridSearchCV(DecisionTreeRegressor(),
                                 param_grid= params_arbol,
                                 cv = 10, # Cross Validation de 5 grupos solo
                                 scoring="neg_mean_squared_error",
                                 n_jobs=-1)

grid_search_arbol.fit(X_train, y_train)
modelo_final_arbol= grid_search_arbol.best_estimator_

In [15]:
modelo_final_arbol.fit(X_train,y_train)
y_pred_test = modelo_final_arbol.predict(X_test)
y_pred_train = modelo_final_arbol.predict(X_train)
df_metricas = obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)
df_metricas

,r2_score,MAE,MSE,RMSE
train,0.44,"2,779.87","122,918,570.39","11,086.86"
test,0.31,"2,801.63","101,312,520.80","10,065.41"


In [16]:
modelo_final_arbol

DecisionTreeRegressor(max_depth=10, max_leaf_nodes=44, min_samples_split=20)

In [17]:
# Atributos del árbol
tree = modelo_final_arbol.tree_

# Identificar hojas
for i in range(tree.node_count):
    if tree.children_left[i] == -1 and tree.children_right[i] == -1:
        print(f"Nodo {i} es una hoja final y tiene {tree.n_node_samples[i]} muestras.")


 
# Obtener el nodo al que pertenece cada fila
nodos_hoja = modelo_final_arbol.apply(X)

# Agrupar las filas según el nodo hoja
for nodo in np.unique(nodos_hoja):
    filas = np.where(nodos_hoja == nodo)[0]  # Índices de las filas en este nodo
    print(f"Nodo {nodo}: {len(filas)} filas -> {filas}")

Nodo 4 es una hoja final y tiene 1 muestras.
Nodo 13 es una hoja final y tiene 12 muestras.
Nodo 16 es una hoja final y tiene 14 muestras.
Nodo 23 es una hoja final y tiene 102040 muestras.
Nodo 25 es una hoja final y tiene 2 muestras.
Nodo 31 es una hoja final y tiene 11138 muestras.
Nodo 32 es una hoja final y tiene 8588 muestras.
Nodo 33 es una hoja final y tiene 1627 muestras.
Nodo 35 es una hoja final y tiene 15706 muestras.
Nodo 37 es una hoja final y tiene 14733 muestras.
Nodo 40 es una hoja final y tiene 1567 muestras.
Nodo 42 es una hoja final y tiene 4179 muestras.
Nodo 43 es una hoja final y tiene 81 muestras.
Nodo 45 es una hoja final y tiene 19 muestras.
Nodo 47 es una hoja final y tiene 631 muestras.
Nodo 51 es una hoja final y tiene 35492 muestras.
Nodo 54 es una hoja final y tiene 186 muestras.
Nodo 55 es una hoja final y tiene 6 muestras.
Nodo 57 es una hoja final y tiene 12 muestras.
Nodo 60 es una hoja final y tiene 11 muestras.
Nodo 61 es una hoja final y tiene 4 mu

# No sigo
Paso al siguiente Modelo
-